# 1️⃣ Setup & Installazione

In [ ]:
# Installazione librerie necessarie
!pip install -q ultralytics opencv-python-headless
!pip install -q matplotlib seaborn pandas numpy pillow

print("✅ Installazione completata!")

In [ ]:
# Import librerie
import os
import json
import csv
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
from tqdm.auto import tqdm

# YOLO
from ultralytics import YOLO
import torch

# Configurazione visualizzazioni
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

# Verifica GPU
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"🎮 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("   ⚠️  GPU non disponibile, uso CPU (più lento)")

print("\n✅ Setup completato!")

# 2️⃣ Caricamento Modello Allenato

## 📦 Opzioni di Caricamento

### Opzione A: Da file locale (Colab o Jupyter)
Se hai già caricato `best.pt` manualmente o è nella cartella `runs/`.

### Opzione B: Da Google Drive (solo Colab)
Se hai salvato i weights su Google Drive durante il training.

### Opzione C: Upload manuale (Colab/Jupyter)
Carica il file `.pt` tramite l'interfaccia.

In [ ]:
# CONFIGURAZIONE PERCORSI

# ========== MODIFICA QUESTI PERCORSI ==========

# Opzione A: Percorso locale (se hai i file già su Colab o localmente)
LOCAL_WEIGHTS_PATH = "runs/detect/optimized_yolov8s/weights/best.pt"

# Opzione B: Percorso su Google Drive (se hai salvato lì)
DRIVE_WEIGHTS_PATH = "/content/drive/MyDrive/cv_project/models/best_detection_optimized.pt"

# Opzione C: Upload manuale - lascia vuoto e usa la cella di upload sotto
MANUAL_UPLOAD_PATH = None  # Verrà settato automaticamente dopo upload

# ============================================

# Scegli quale usare (cambia questo valore)
USE_SOURCE = "local"  # Opzioni: "local", "drive", "upload"

print(f"✅ Configurazione caricamento: {USE_SOURCE.upper()}")

In [ ]:
# SOLO SE USE_SOURCE = "drive": Mount Google Drive

if USE_SOURCE == "drive":
    try:
        from google.colab import auth, drive
        print("📁 Mounting Google Drive...")
        auth.authenticate_user()
        drive.mount('/content/drive')
        print("✅ Google Drive montato!")
    except ImportError:
        print("⚠️  Non sei su Google Colab. Cambia USE_SOURCE a 'local' o 'upload'.")
else:
    print("ℹ️  Google Drive mount non necessario (non stai usando 'drive').")

In [ ]:
# SOLO SE USE_SOURCE = "upload": Upload manuale del file .pt

if USE_SOURCE == "upload":
    try:
        from google.colab import files
        print("📤 Carica il file best.pt dal tuo computer:")
        uploaded = files.upload()
        
        # Trova il file .pt caricato
        pt_files = [f for f in uploaded.keys() if f.endswith('.pt')]
        if pt_files:
            MANUAL_UPLOAD_PATH = pt_files[0]
            print(f"✅ File caricato: {MANUAL_UPLOAD_PATH}")
        else:
            print("❌ Nessun file .pt trovato nell'upload!")
    except ImportError:
        print("⚠️  Non sei su Colab. Usa Jupyter upload widget o cambia USE_SOURCE a 'local'.")
        # Per Jupyter locale, puoi usare widget upload:
        # from IPython.display import FileUpload
        # display(FileUpload())
else:
    print("ℹ️  Upload manuale non necessario (non stai usando 'upload').")

In [ ]:
# CARICAMENTO MODELLO

print("🤖 Caricamento modello YOLO...\n")
print("="*60)

# Determina quale path usare
if USE_SOURCE == "local":
    weights_path = LOCAL_WEIGHTS_PATH
elif USE_SOURCE == "drive":
    weights_path = DRIVE_WEIGHTS_PATH
elif USE_SOURCE == "upload":
    if MANUAL_UPLOAD_PATH:
        weights_path = MANUAL_UPLOAD_PATH
    else:
        raise ValueError("MANUAL_UPLOAD_PATH non settato. Esegui prima la cella di upload.")
else:
    raise ValueError(f"USE_SOURCE non valido: {USE_SOURCE}")

# Verifica esistenza file
weights_path = Path(weights_path)
if not weights_path.exists():
    raise FileNotFoundError(
        f"❌ File non trovato: {weights_path}\n"
        f"   Verifica il percorso o cambia USE_SOURCE."
    )

print(f"📂 Percorso weights: {weights_path}")
print(f"📦 Dimensione file: {weights_path.stat().st_size / 1024**2:.2f} MB\n")

# Carica modello
model = YOLO(str(weights_path))

# Device (GPU se disponibile)
device = 0 if torch.cuda.is_available() else 'cpu'

print(f"✅ Modello caricato con successo!")
print(f"🎮 Device: {device}")
print(f"🎯 Numero di classi: {len(model.names)}")
print(f"📋 Classi: {list(model.names.values())}")
print("="*60)

# 3️⃣ Predizione su Singola Immagine

Esempio base: carica un'immagine e predici.

In [ ]:
# CONFIGURAZIONE: Path dell'immagine di test

# ========== MODIFICA QUESTO PERCORSO ==========
TEST_IMAGE_PATH = "path/to/your/test_image.jpg"  # Cambia con il tuo percorso

# Oppure usa upload manuale (decommenta la cella sotto)
# ============================================

print(f"📸 Immagine di test: {TEST_IMAGE_PATH}")

In [ ]:
# OPZIONALE: Upload manuale immagine di test (solo Colab)

# Decommenta se vuoi caricare un'immagine dal tuo computer
# try:
#     from google.colab import files
#     print("📤 Carica un'immagine di test:")
#     uploaded = files.upload()
#     TEST_IMAGE_PATH = list(uploaded.keys())[0]
#     print(f"✅ Immagine caricata: {TEST_IMAGE_PATH}")
# except ImportError:
#     print("⚠️  Non sei su Colab, usa il path locale sopra.")

print("ℹ️  Se vuoi usare upload, decommenta il codice in questa cella.")

In [ ]:
# PREDIZIONE SU SINGOLA IMMAGINE

print("🔮 PREDIZIONE SU SINGOLA IMMAGINE\n")
print("="*60)

# Verifica esistenza immagine
test_img_path = Path(TEST_IMAGE_PATH)
if not test_img_path.exists():
    print(f"❌ Immagine non trovata: {test_img_path}")
    print("   Modifica TEST_IMAGE_PATH nella cella sopra o usa upload.")
else:
    # Predizione
    print(f"📂 Immagine: {test_img_path.name}")
    print(f"🔍 Esecuzione predizione...\n")
    
    results = model.predict(
        source=str(test_img_path),
        conf=0.25,           # Confidence threshold
        iou=0.6,             # NMS IoU threshold
        device=device,
        verbose=False
    )
    
    result = results[0]  # Un solo risultato (una immagine)
    
    # Info detection
    n_detections = len(result.boxes)
    print(f"✅ Predizione completata!")
    print(f"📊 Oggetti rilevati: {n_detections}\n")
    
    if n_detections > 0:
        print("📋 Dettagli detection:\n")
        for idx, box in enumerate(result.boxes, 1):
            cls_id = int(box.cls.cpu().numpy()[0])
            cls_name = model.names[cls_id]
            conf = float(box.conf.cpu().numpy()[0])
            xyxy = box.xyxy.cpu().numpy()[0]  # [x1, y1, x2, y2]
            print(f"   {idx}. {cls_name:12s} | Confidence: {conf:.3f} | BBox: [{xyxy[0]:.0f}, {xyxy[1]:.0f}, {xyxy[2]:.0f}, {xyxy[3]:.0f}]")
    else:
        print("⚠️  Nessun oggetto rilevato (prova ad abbassare conf threshold).")
    
    # Visualizzazione
    print("\n🖼️  Visualizzazione immagine annotata:\n")
    
    annotated_img = result.plot()  # BGR array
    annotated_img_rgb = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(12, 8))
    plt.imshow(annotated_img_rgb)
    plt.axis('off')
    plt.title(f"{test_img_path.name} - {n_detections} oggetti rilevati", fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("="*60)

# 4️⃣ Predizione su Batch di Immagini

Esegui inferenza su un'intera cartella di immagini.

In [ ]:
# CONFIGURAZIONE: Cartella con immagini di test

# ========== MODIFICA QUESTI PERCORSI ==========
TEST_IMAGES_FOLDER = "path/to/test_images_folder"  # Cartella con le immagini
OUTPUT_FOLDER = "inference_results"                 # Dove salvare i risultati
# ============================================

print(f"📁 Cartella test images: {TEST_IMAGES_FOLDER}")
print(f"💾 Cartella output: {OUTPUT_FOLDER}")

In [ ]:
# PREDIZIONE BATCH

print("🔮 PREDIZIONE BATCH\n")
print("="*60)

test_folder = Path(TEST_IMAGES_FOLDER)
output_folder = Path(OUTPUT_FOLDER)

# Verifica cartella
if not test_folder.exists():
    print(f"❌ Cartella non trovata: {test_folder}")
    print("   Modifica TEST_IMAGES_FOLDER nella cella sopra.")
else:
    # Trova tutte le immagini
    image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.tiff']
    test_images = []
    for ext in image_extensions:
        test_images.extend(test_folder.glob(ext))
        test_images.extend(test_folder.glob(ext.upper()))
    
    test_images = sorted(set(test_images))  # Rimuovi duplicati
    
    print(f"📊 Trovate {len(test_images)} immagini in {test_folder.name}\n")
    
    if len(test_images) == 0:
        print("⚠️  Nessuna immagine trovata nella cartella!")
    else:
        # Crea cartella output
        output_folder.mkdir(parents=True, exist_ok=True)
        output_images_folder = output_folder / "images"
        output_images_folder.mkdir(exist_ok=True)
        
        print(f"🔍 Esecuzione predizione batch...\n")
        
        # Predizione batch
        # YOLO supporta batch automatico
        results = model.predict(
            source=str(test_folder),
            conf=0.25,
            iou=0.6,
            device=device,
            save=True,            # Salva immagini annotate
            project=str(output_folder),
            name="predictions",
            exist_ok=True,
            verbose=True
        )
        
        print(f"\n✅ Predizione batch completata!")
        print(f"💾 Risultati salvati in: {output_folder / 'predictions'}")
        print(f"📊 Processate {len(results)} immagini")
        
        # Statistiche
        total_detections = sum(len(r.boxes) for r in results)
        print(f"\n📈 STATISTICHE:")
        print(f"   Totale detection: {total_detections}")
        print(f"   Media per immagine: {total_detections/len(results):.2f}")
        
        # Conta per classe
        class_counts = {}
        for r in results:
            for box in r.boxes:
                cls_id = int(box.cls.cpu().numpy()[0])
                cls_name = model.names[cls_id]
                class_counts[cls_name] = class_counts.get(cls_name, 0) + 1
        
        print(f"\n📊 Detection per classe:")
        for cls_name, count in sorted(class_counts.items(), key=lambda x: -x[1]):
            print(f"   {cls_name:12s}: {count:4d}")
        
        print("\n" + "="*60)

# 5️⃣ Estrazione Programmatica dei Risultati

Estrai bbox, confidence e classi in formato strutturato (CSV/JSON) per analisi successive.

In [ ]:
# ESTRAZIONE RISULTATI IN FORMATO STRUTTURATO

print("📊 ESTRAZIONE RISULTATI\n")
print("="*60)

# Assicurati di aver eseguito prima la predizione batch sopra
if 'results' not in locals():
    print("⚠️  Esegui prima la cella di predizione batch.")
else:
    # Lista per raccogliere tutte le detection
    all_detections = []
    
    for result in results:
        img_name = Path(result.path).name
        img_width = result.orig_shape[1]
        img_height = result.orig_shape[0]
        
        for box in result.boxes:
            # Estrai info
            xyxy = box.xyxy.cpu().numpy()[0]  # [x1, y1, x2, y2]
            xywh = box.xywh.cpu().numpy()[0]  # [x_center, y_center, width, height]
            conf = float(box.conf.cpu().numpy()[0])
            cls_id = int(box.cls.cpu().numpy()[0])
            cls_name = model.names[cls_id]
            
            # Crea record
            detection = {
                'image': img_name,
                'class_id': cls_id,
                'class_name': cls_name,
                'confidence': conf,
                'bbox_x1': float(xyxy[0]),
                'bbox_y1': float(xyxy[1]),
                'bbox_x2': float(xyxy[2]),
                'bbox_y2': float(xyxy[3]),
                'bbox_x_center': float(xywh[0]),
                'bbox_y_center': float(xywh[1]),
                'bbox_width': float(xywh[2]),
                'bbox_height': float(xywh[3]),
                'image_width': img_width,
                'image_height': img_height
            }
            
            all_detections.append(detection)
    
    # Converti in DataFrame
    df_detections = pd.DataFrame(all_detections)
    
    print(f"✅ Estratte {len(df_detections)} detection da {len(results)} immagini\n")
    
    # Mostra anteprima
    print("📋 Anteprima dati (prime 10 righe):\n")
    print(df_detections.head(10).to_string(index=False))
    
    # Salva CSV
    csv_path = output_folder / "detections.csv"
    df_detections.to_csv(csv_path, index=False)
    print(f"\n💾 CSV salvato: {csv_path}")
    
    # Salva JSON
    json_path = output_folder / "detections.json"
    with open(json_path, 'w') as f:
        json.dump(all_detections, f, indent=2)
    print(f"💾 JSON salvato: {json_path}")
    
    print("\n" + "="*60)

# 6️⃣ Visualizzazioni Avanzate

Grafici e analisi sui risultati delle predizioni.

In [ ]:
# VISUALIZZAZIONI: Distribuzione confidence e classi

if 'df_detections' not in locals():
    print("⚠️  Esegui prima l'estrazione risultati.")
else:
    print("📊 VISUALIZZAZIONI\n")
    print("="*60)
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. Distribuzione confidence
    axes[0, 0].hist(df_detections['confidence'], bins=30, edgecolor='black', alpha=0.7)
    axes[0, 0].set_xlabel('Confidence')
    axes[0, 0].set_ylabel('Frequenza')
    axes[0, 0].set_title('Distribuzione Confidence Score')
    axes[0, 0].axvline(df_detections['confidence'].mean(), color='red', 
                      linestyle='--', label=f'Media: {df_detections["confidence"].mean():.3f}')
    axes[0, 0].legend()
    axes[0, 0].grid(alpha=0.3)
    
    # 2. Conteggio per classe
    class_counts = df_detections['class_name'].value_counts()
    axes[0, 1].bar(range(len(class_counts)), class_counts.values, 
                  color=plt.cm.Set3(range(len(class_counts))))
    axes[0, 1].set_xticks(range(len(class_counts)))
    axes[0, 1].set_xticklabels(class_counts.index, rotation=45, ha='right')
    axes[0, 1].set_ylabel('Numero Detection')
    axes[0, 1].set_title('Detection per Classe')
    axes[0, 1].grid(axis='y', alpha=0.3)
    
    # 3. Boxplot confidence per classe
    df_detections.boxplot(column='confidence', by='class_name', ax=axes[1, 0])
    axes[1, 0].set_xlabel('Classe')
    axes[1, 0].set_ylabel('Confidence')
    axes[1, 0].set_title('Confidence per Classe')
    axes[1, 0].get_figure().suptitle('')  # Rimuovi titolo auto
    plt.setp(axes[1, 0].xaxis.get_majorticklabels(), rotation=45, ha='right')
    
    # 4. Distribuzione dimensioni bbox
    df_detections['bbox_area'] = df_detections['bbox_width'] * df_detections['bbox_height']
    axes[1, 1].scatter(df_detections['bbox_width'], df_detections['bbox_height'], 
                      alpha=0.5, s=20)
    axes[1, 1].set_xlabel('BBox Width (px)')
    axes[1, 1].set_ylabel('BBox Height (px)')
    axes[1, 1].set_title('Distribuzione Dimensioni BBox')
    axes[1, 1].grid(alpha=0.3)
    
    plt.tight_layout()
    viz_path = output_folder / "visualizations.png"
    plt.savefig(viz_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"💾 Visualizzazioni salvate: {viz_path}")
    print("="*60)

In [ ]:
# VISUALIZZAZIONE: Griglia di immagini predette

if 'results' not in locals():
    print("⚠️  Esegui prima la predizione batch.")
else:
    print("🖼️  GRIGLIA IMMAGINI PREDETTE\n")
    print("="*60)
    
    # Seleziona un campione casuale
    import random
    sample_size = min(12, len(results))  # Max 12 immagini
    sample_results = random.sample(results, sample_size)
    
    # Crea griglia
    cols = 4
    rows = (sample_size + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(20, 5*rows))
    axes = axes.flatten() if sample_size > 1 else [axes]
    
    for idx, result in enumerate(sample_results):
        annotated_img = result.plot()
        annotated_img_rgb = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)
        
        axes[idx].imshow(annotated_img_rgb)
        axes[idx].axis('off')
        axes[idx].set_title(
            f"{Path(result.path).name}\n{len(result.boxes)} oggetti",
            fontsize=10
        )
    
    # Nascondi assi vuoti
    for idx in range(sample_size, len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    grid_path = output_folder / "sample_grid.png"
    plt.savefig(grid_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"💾 Griglia salvata: {grid_path}")
    print("="*60)

# 7️⃣ Riepilogo e Prossimi Passi

## ✅ Cosa Abbiamo Fatto

1. ✅ Installato librerie necessarie
2. ✅ Caricato modello YOLO allenato (da locale/Drive/upload)
3. ✅ Predetto su singola immagine con visualizzazione
4. ✅ Predetto su batch di immagini
5. ✅ Estratto risultati in CSV/JSON strutturati
6. ✅ Creato visualizzazioni e statistiche

---

## 🚀 Prossimi Passi Possibili

### 1. Deploy come API
```python
# Usa FastAPI o Flask per creare un'API REST
from fastapi import FastAPI, File, UploadFile

app = FastAPI()

@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    # Carica immagine
    # Esegui model.predict()
    # Ritorna JSON con detection
    pass
```

### 2. Conversione per Mobile/Edge
```python
# Export a ONNX
model.export(format='onnx')

# Export a TensorFlow Lite
model.export(format='tflite')

# Export a CoreML (iOS)
model.export(format='coreml')
```

### 3. Video Processing
```python
# Predizione su video
results = model.predict(
    source='video.mp4',
    save=True,
    stream=True  # Generator per processare frame-by-frame
)
```

### 4. Tracking Multi-Object
```python
# YOLO supporta tracking integrato
results = model.track(
    source='video.mp4',
    tracker='bytetrack.yaml'  # o 'botsort.yaml'
)
```

### 5. Fine-tuning Continuo
```python
# Riprendi training da checkpoint
model = YOLO('best.pt')
model.train(
    data='new_data.yaml',
    epochs=20,
    resume=True  # Continua da best.pt
)
```

---

## 📚 Risorse Utili

- **Ultralytics Docs**: https://docs.ultralytics.com
- **Export Formats**: https://docs.ultralytics.com/modes/export/
- **Deployment Guide**: https://docs.ultralytics.com/guides/

---

## 💡 Consigli

1. **Threshold Tuning**: Prova confidence threshold diversi (0.1-0.5) per bilanciare precision/recall
2. **NMS Tuning**: Regola IoU threshold per gestire sovrapposizioni
3. **Batch Size**: Per inferenza veloce su molte immagini, aumenta batch size
4. **Augmentation Test-Time**: Prova TTA (Test-Time Augmentation) per accuracy maggiore:
   ```python
   results = model.predict(source=img, augment=True)
   ```

In [ ]:

# Import librerie
import os
import json
import csv
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
from tqdm.auto import tqdm

from ultralytics import YOLO
import torch

plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

print(f"Versione PyTorch: {torch.__version__}")
print(f"CUDA disponibile: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("GPU non disponibile, uso CPU")

print("Setup completato!")

# Configurazione caricamento modello

LOCAL_WEIGHTS_PATH = "runs/detect/optimized_yolov8s/weights/best.pt"
DRIVE_WEIGHTS_PATH = "/content/drive/MyDrive/cv_project/models/best_detection_optimized.pt"
MANUAL_UPLOAD_PATH = None

USE_SOURCE = "local"

print(f"Configurazione: {USE_SOURCE.upper()}")

# Mount Google Drive se necessario

if USE_SOURCE == "drive":
    try:
        from google.colab import auth, drive
        print("Mount Google Drive...")
        auth.authenticate_user()
        drive.mount('/content/drive')
        print("Google Drive montato!")
    except ImportError:
        print("Non sei su Google Colab. Cambia USE_SOURCE a 'local' o 'upload'.")
else:
    print("Google Drive mount non necessario.")

# Upload manuale file .pt

if USE_SOURCE == "upload":
    try:
        from google.colab import files
        print("Carica il file best.pt:")
        uploaded = files.upload()
        
        pt_files = [f for f in uploaded.keys() if f.endswith('.pt')]
        if pt_files:
            MANUAL_UPLOAD_PATH = pt_files[0]
            print(f"File caricato: {MANUAL_UPLOAD_PATH}")
        else:
            print("Nessun file .pt trovato!")
    except ImportError:
        print("Non sei su Colab. Usa path locale o cambia USE_SOURCE.")
else:
    print("Upload manuale non necessario.")

# Caricamento modello

print("Caricamento modello YOLO...\n")
print("="*60)

if USE_SOURCE == "local":
    weights_path = LOCAL_WEIGHTS_PATH
elif USE_SOURCE == "drive":
    weights_path = DRIVE_WEIGHTS_PATH
elif USE_SOURCE == "upload":
    if MANUAL_UPLOAD_PATH:
        weights_path = MANUAL_UPLOAD_PATH
    else:
        raise ValueError("MANUAL_UPLOAD_PATH non settato.")
else:
    raise ValueError(f"USE_SOURCE non valido: {USE_SOURCE}")

weights_path = Path(weights_path)
if not weights_path.exists():
    raise FileNotFoundError(f"File non trovato: {weights_path}")

print(f"Percorso weights: {weights_path}")
print(f"Dimensione file: {weights_path.stat().st_size / 1024**2:.2f} MB\n")

model = YOLO(str(weights_path))
device = 0 if torch.cuda.is_available() else 'cpu'

print(f"Modello caricato con successo!")
print(f"Device: {device}")
print(f"Numero di classi: {len(model.names)}")
print(f"Classi: {list(model.names.values())}")
print("="*60)

# Predizione su singola immagine

TEST_IMAGE_PATH = "path/to/your/test_image.jpg"

print(f"Immagine di test: {TEST_IMAGE_PATH}")

# Upload manuale immagine (opzionale)

# try:
#     from google.colab import files
#     print("Carica un'immagine di test:")
#     uploaded = files.upload()
#     TEST_IMAGE_PATH = list(uploaded.keys())[0]
#     print(f"Immagine caricata: {TEST_IMAGE_PATH}")
# except ImportError:
#     print("Non sei su Colab, usa path locale.")

print("Se vuoi usare upload, decommenta il codice sopra.")

# Esecuzione predizione singola immagine

print("PREDIZIONE SU SINGOLA IMMAGINE\n")
print("="*60)

test_img_path = Path(TEST_IMAGE_PATH)
if not test_img_path.exists():
    print(f"Immagine non trovata: {test_img_path}")
    print("Modifica TEST_IMAGE_PATH o usa upload.")
else:
    print(f"Immagine: {test_img_path.name}")
    print(f"Esecuzione predizione...\n")
    
    results = model.predict(
        source=str(test_img_path),
        conf=0.25,
        iou=0.6,
        device=device,
        verbose=False
    )
    
    result = results[0]
    n_detections = len(result.boxes)
    
    print(f"Predizione completata!")
    print(f"Oggetti rilevati: {n_detections}\n")
    
    if n_detections > 0:
        print("Dettagli detection:\n")
        for idx, box in enumerate(result.boxes, 1):
            cls_id = int(box.cls.cpu().numpy()[0])
            cls_name = model.names[cls_id]
            conf = float(box.conf.cpu().numpy()[0])
            xyxy = box.xyxy.cpu().numpy()[0]
            print(f"   {idx}. {cls_name:12s} | Confidence: {conf:.3f} | BBox: [{xyxy[0]:.0f}, {xyxy[1]:.0f}, {xyxy[2]:.0f}, {xyxy[3]:.0f}]")
    else:
        print("Nessun oggetto rilevato (prova ad abbassare conf threshold).")
    
    print("\nVisualizzazione immagine annotata:\n")
    
    annotated_img = result.plot()
    annotated_img_rgb = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(12, 8))
    plt.imshow(annotated_img_rgb)
    plt.axis('off')
    plt.title(f"{test_img_path.name} - {n_detections} oggetti rilevati", fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("="*60)

# Predizione batch

TEST_IMAGES_FOLDER = "path/to/test_images_folder"
OUTPUT_FOLDER = "inference_results"

print(f"Cartella test images: {TEST_IMAGES_FOLDER}")
print(f"Cartella output: {OUTPUT_FOLDER}")

# Esecuzione predizione batch

print("PREDIZIONE BATCH\n")
print("="*60)

test_folder = Path(TEST_IMAGES_FOLDER)
output_folder = Path(OUTPUT_FOLDER)

if not test_folder.exists():
    print(f"Cartella non trovata: {test_folder}")
    print("Modifica TEST_IMAGES_FOLDER.")
else:
    image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.tiff']
    test_images = []
    for ext in image_extensions:
        test_images.extend(test_folder.glob(ext))
        test_images.extend(test_folder.glob(ext.upper()))
    
    test_images = sorted(set(test_images))
    
    print(f"Trovate {len(test_images)} immagini in {test_folder.name}\n")
    
    if len(test_images) == 0:
        print("Nessuna immagine trovata nella cartella!")
    else:
        output_folder.mkdir(parents=True, exist_ok=True)
        output_images_folder = output_folder / "images"
        output_images_folder.mkdir(exist_ok=True)
        
        print(f"Esecuzione predizione batch...\n")
        
        results = model.predict(
            source=str(test_folder),
            conf=0.25,
            iou=0.6,
            device=device,
            save=True,
            project=str(output_folder),
            name="predictions",
            exist_ok=True,
            verbose=True
        )
        
        print(f"\nPredizione batch completata!")
        print(f"Risultati salvati in: {output_folder / 'predictions'}")
        print(f"Processate {len(results)} immagini")
        
        total_detections = sum(len(r.boxes) for r in results)
        print(f"\nSTATISTICHE:")
        print(f"   Totale detection: {total_detections}")
        print(f"   Media per immagine: {total_detections/len(results):.2f}")
        
        class_counts = {}
        for r in results:
            for box in r.boxes:
                cls_id = int(box.cls.cpu().numpy()[0])
                cls_name = model.names[cls_id]
                class_counts[cls_name] = class_counts.get(cls_name, 0) + 1
        
        print(f"\nDetection per classe:")
        for cls_name, count in sorted(class_counts.items(), key=lambda x: -x[1]):
            print(f"   {cls_name:12s}: {count:4d}")
        
        print("\n" + "="*60)

# Estrazione risultati in formato strutturato

print("ESTRAZIONE RISULTATI\n")
print("="*60)

if 'results' not in locals():
    print("Esegui prima la predizione batch.")
else:
    all_detections = []
    
    for result in results:
        img_name = Path(result.path).name
        img_width = result.orig_shape[1]
        img_height = result.orig_shape[0]
        
        for box in result.boxes:
            xyxy = box.xyxy.cpu().numpy()[0]
            xywh = box.xywh.cpu().numpy()[0]
            conf = float(box.conf.cpu().numpy()[0])
            cls_id = int(box.cls.cpu().numpy()[0])
            cls_name = model.names[cls_id]
            
            detection = {
                'image': img_name,
                'class_id': cls_id,
                'class_name': cls_name,
                'confidence': conf,
                'bbox_x1': float(xyxy[0]),
                'bbox_y1': float(xyxy[1]),
                'bbox_x2': float(xyxy[2]),
                'bbox_y2': float(xyxy[3]),
                'bbox_x_center': float(xywh[0]),
                'bbox_y_center': float(xywh[1]),
                'bbox_width': float(xywh[2]),
                'bbox_height': float(xywh[3]),
                'image_width': img_width,
                'image_height': img_height
            }
            
            all_detections.append(detection)
    
    df_detections = pd.DataFrame(all_detections)
    
    print(f"Estratte {len(df_detections)} detection da {len(results)} immagini\n")
    
    print("Anteprima dati (prime 10 righe):\n")
    print(df_detections.head(10).to_string(index=False))
    
    csv_path = output_folder / "detections.csv"
    df_detections.to_csv(csv_path, index=False)
    print(f"\nCSV salvato: {csv_path}")
    
    json_path = output_folder / "detections.json"
    with open(json_path, 'w') as f:
        json.dump(all_detections, f, indent=2)
    print(f"JSON salvato: {json_path}")
    
    print("\n" + "="*60)

# Visualizzazioni

if 'df_detections' not in locals():
    print("Esegui prima l'estrazione risultati.")
else:
    print("VISUALIZZAZIONI\n")
    print("="*60)
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    axes[0, 0].hist(df_detections['confidence'], bins=30, edgecolor='black', alpha=0.7)
    axes[0, 0].set_xlabel('Confidence')
    axes[0, 0].set_ylabel('Frequenza')
    axes[0, 0].set_title('Distribuzione Confidence Score')
    axes[0, 0].axvline(df_detections['confidence'].mean(), color='red', 
                      linestyle='--', label=f'Media: {df_detections["confidence"].mean():.3f}')
    axes[0, 0].legend()
    axes[0, 0].grid(alpha=0.3)
    
    class_counts = df_detections['class_name'].value_counts()
    axes[0, 1].bar(range(len(class_counts)), class_counts.values, 
                  color=plt.cm.Set3(range(len(class_counts))))
    axes[0, 1].set_xticks(range(len(class_counts)))
    axes[0, 1].set_xticklabels(class_counts.index, rotation=45, ha='right')
    axes[0, 1].set_ylabel('Numero Detection')
    axes[0, 1].set_title('Detection per Classe')
    axes[0, 1].grid(axis='y', alpha=0.3)
    
    df_detections.boxplot(column='confidence', by='class_name', ax=axes[1, 0])
    axes[1, 0].set_xlabel('Classe')
    axes[1, 0].set_ylabel('Confidence')
    axes[1, 0].set_title('Confidence per Classe')
    axes[1, 0].get_figure().suptitle('')
    plt.setp(axes[1, 0].xaxis.get_majorticklabels(), rotation=45, ha='right')
    
    df_detections['bbox_area'] = df_detections['bbox_width'] * df_detections['bbox_height']
    axes[1, 1].scatter(df_detections['bbox_width'], df_detections['bbox_height'], 
                      alpha=0.5, s=20)
    axes[1, 1].set_xlabel('BBox Width (px)')
    axes[1, 1].set_ylabel('BBox Height (px)')
    axes[1, 1].set_title('Distribuzione Dimensioni BBox')
    axes[1, 1].grid(alpha=0.3)
    
    plt.tight_layout()
    viz_path = output_folder / "visualizations.png"
    plt.savefig(viz_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"Visualizzazioni salvate: {viz_path}")
    print("="*60)

# Griglia immagini predette

if 'results' not in locals():
    print("Esegui prima la predizione batch.")
else:
    print("GRIGLIA IMMAGINI PREDETTE\n")
    print("="*60)
    
    import random
    sample_size = min(12, len(results))
    sample_results = random.sample(results, sample_size)
    
    cols = 4
    rows = (sample_size + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(20, 5*rows))
    axes = axes.flatten() if sample_size > 1 else [axes]
    
    for idx, result in enumerate(sample_results):
        annotated_img = result.plot()
        annotated_img_rgb = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)
        
        axes[idx].imshow(annotated_img_rgb)
        axes[idx].axis('off')
        axes[idx].set_title(
            f"{Path(result.path).name}\n{len(result.boxes)} oggetti",
            fontsize=10
        )
    
    for idx in range(sample_size, len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    grid_path = output_folder / "sample_grid.png"
    plt.savefig(grid_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"Griglia salvata: {grid_path}")
    print("="*60)